In [45]:
pip install scipy

     ---------------------------------------- 0.0/60.8 kB ? eta -:--:--
     ------ --------------------------------- 10.2/60.8 kB ? eta -:--:--
     ------------------- ------------------ 30.7/60.8 kB 262.6 kB/s eta 0:00:01
     -------------------------------- ----- 51.2/60.8 kB 327.7 kB/s eta 0:00:01
     -------------------------------------- 60.8/60.8 kB 323.9 kB/s eta 0:00:00
   ---------------------------------------- 0.0/44.5 MB ? eta -:--:--
   ---------------------------------------- 0.1/44.5 MB 3.2 MB/s eta 0:00:14
   ---------------------------------------- 0.1/44.5 MB 1.3 MB/s eta 0:00:34
   ---------------------------------------- 0.2/44.5 MB 1.6 MB/s eta 0:00:28
   ---------------------------------------- 0.2/44.5 MB 1.6 MB/s eta 0:00:28
   ---------------------------------------- 0.5/44.5 MB 2.0 MB/s eta 0:00:22
   ---------------------------------------- 0.5/44.5 MB 1.9 MB/s eta 0:00:24
    --------------------------------------- 1.0/44.5 MB 3.0 MB/s eta 0:00:15
   - -


[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [44]:
# Importing Necessary Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from scipy.stats import zscore

# Set visual styles
sns.set(style="whitegrid")
plt.style.use("fivethirtyeight")


ModuleNotFoundError: No module named 'scipy'

In [10]:
telemetry1 = pd.read_csv("telemetry_1.csv")
telemetry2 = pd.read_csv("telemetry_2.csv")
vehicle_data = pd.read_csv("vehicle_data.csv")

## Display the first few rows of each dataset to understand the structure

In [11]:
print("Telemetry 1:\n", telemetry1.head())
print("\nTelemetry 2:\n", telemetry2.head())
print("\nVehicle Data:\n", vehicle_data.head())

Telemetry 1:
    vehicle_id                         timestamp  speed  odometer  fuel_level
0   vehicle_4         2024-06-06 14:26:23-07:00   64.2       NaN         NaN
1   vehicle_8  2024-07-05 22:24:12.856000-07:00    1.6   12048.0         NaN
2   vehicle_3  2024-06-20 12:57:59.306000-07:00    NaN       NaN        45.0
3  vehicle_10  2024-07-24 18:04:23.776000-07:00   36.2    1434.7         NaN
4  vehicle_11         2024-07-06 11:08:03-07:00   29.5       NaN         NaN

Telemetry 2:
    vehicle_id                         timestamp   val   name
0   vehicle_4         2024-07-23 19:56:44+05:30   0.0  speed
1  vehicle_11         2024-04-04 22:33:56+05:30  52.7  speed
2   vehicle_3  2024-07-18 23:30:04.027000+05:30   5.8  speed
3   vehicle_3  2024-05-22 00:49:49.402000+05:30  34.4  speed
4  vehicle_10  2024-07-16 05:48:11.710000+05:30  42.8  speed

Vehicle Data:
           id datasource  tank_capacity  rated_mpg
0  vehicle_1   source_1           18.0         35
1  vehicle_2   source_1    

# Normalize Telemetry 2 by pivoting so that the 'name' column becomes the new columns

In [16]:
telemetry2_pivot = telemetry2.pivot_table(index=['vehicle_id', 'timestamp'], columns='name', values='val').reset_index()

In [17]:
telemetry_combined = pd.concat([telemetry1, telemetry2_pivot], axis=0, ignore_index=True)

In [24]:
fuel_economy = []
for vehicle_id in telemetry_combined['vehicle_id'].unique():
    vehicle_telemetry = telemetry_combined[telemetry_combined['vehicle_id'] == vehicle_id].sort_values('timestamp')
    start_odometer = vehicle_telemetry['odometer'].iloc[0]
    end_odometer = vehicle_telemetry['odometer'].iloc[-1]
    total_miles = end_odometer - start_odometer
    
    start_fuel = vehicle_telemetry['fuel_level'].iloc[0]
    end_fuel = vehicle_telemetry['fuel_level'].iloc[-1]
    tank_capacity = vehicle_data.loc[vehicle_data['id'] == vehicle_id, 'tank_capacity'].values[0]
    fuel_consumed = (start_fuel - end_fuel) / 100 * tank_capacity
    
    mpg = total_miles / fuel_consumed
    fuel_economy.append({'Vehicle ID': vehicle_id, 'Calculated MPG': mpg})

fuel_economy_df = pd.DataFrame(fuel_economy)

In [31]:
print(vehicle_data.columns)


Index(['id', 'datasource', 'tank_capacity', 'rated_mpg'], dtype='object')


In [36]:
result = pd.merge(fuel_economy_df, vehicle_data[['id', 'rated_mpg']], left_on='Vehicle ID', right_on='id')

In [37]:
result = result.drop(columns=['id'])

In [38]:
result = pd.merge(fuel_economy_df, vehicle_data[['id', 'rated_mpg']], left_on='Vehicle ID', right_on='id')
result = result.drop(columns=['id'])

print("\nFuel Economy Results:\n", result)



Fuel Economy Results:
     Vehicle ID  Calculated MPG  rated_mpg
0    vehicle_4    15817.307692         32
1    vehicle_8             NaN         21
2    vehicle_3             NaN         24
3   vehicle_10             NaN         21
4   vehicle_11             NaN         35
5    vehicle_5             NaN         22
6    vehicle_7             NaN         21
7    vehicle_1             NaN         35
8    vehicle_6             NaN         56
9    vehicle_9             NaN         24
10   vehicle_2             NaN         32
